In [4]:
import clawpack_annular as cpa

In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
from clawpack import pyclaw
from clawpack.pyclaw.examples.advection_2d_annulus.mapc2p import mapc2p

import numpy as np
from clawpack.visclaw import colormaps

Use parts of `run_app_from_main` below (https://github.com/clawpack/pyclaw/blob/151aefa92a613d952f3b49070cd80826a81b1178/src/pyclaw/util.py#L83)

In [12]:
# these are usually passed to setup function.
pyclaw_kwargs = {'use_petsc':False,
                'outdir':'./_output',
                'solver_type':'classic'}

In [9]:
clawSolution = cpa.setup()

status = clawSolution.run()
status

2024-05-22 23:21:01,339 INFO CLAW: Solution 0 computed for time t=0.000000
2024-05-22 23:21:01,391 INFO CLAW: Solution 1 computed for time t=0.100000
2024-05-22 23:21:01,431 INFO CLAW: Solution 2 computed for time t=0.200000
2024-05-22 23:21:01,471 INFO CLAW: Solution 3 computed for time t=0.300000
2024-05-22 23:21:01,511 INFO CLAW: Solution 4 computed for time t=0.400000
2024-05-22 23:21:01,552 INFO CLAW: Solution 5 computed for time t=0.500000
2024-05-22 23:21:01,593 INFO CLAW: Solution 6 computed for time t=0.600000
2024-05-22 23:21:01,633 INFO CLAW: Solution 7 computed for time t=0.700000
2024-05-22 23:21:01,673 INFO CLAW: Solution 8 computed for time t=0.800000
2024-05-22 23:21:01,713 INFO CLAW: Solution 9 computed for time t=0.900000
2024-05-22 23:21:01,753 INFO CLAW: Solution 10 computed for time t=1.000000


{'cflmax': 0.4000000000000001,
 'dtmin': 0.0033318104550021824,
 'dtmax': 0.0033318104550021832,
 'numsteps': 310}

In [15]:
outdir = pyclaw_kwargs.get('outdir','./_output')
htmlplot = pyclaw_kwargs.get('htmlplot',False) # return False if key doesn't exist
iplot    = pyclaw_kwargs.get('iplot',False)
outdir, htmlplot, iplot

('./_output', False, False)

In [19]:
pyclaw.plot.html_plot(outdir=outdir,setplot=cpa.setplot)

/home/ajivani/clawpack_src/clawpack/visclaw/src/python/visclaw/plotpages.py:605: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if plotdata.format is 'forestclaw':
/home/ajivani/clawpack_src/clawpack/visclaw/src/python/visclaw/plotpages.py:1809: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if prefix is 'fort':


Executed setplot successfully
Will plot 11 frames numbered: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Will make 2 figure(s) for each frame, numbered:  [0, 1]

-----------------------------------


Creating html pages for figures...

Directory '/home/ajivani/AdvectionWL/_plots' 
    already exists, files may be overwritten 
Now making png files for all figures...
    Reading  Frame 0 at t = 0  from outdir = /home/ajivani/AdvectionWL/_output
Frame 0 at time t = 0.0
    Reading  Frame 1 at t = 0.1  from outdir = /home/ajivani/AdvectionWL/_output
Frame 1 at time t = 0.1
    Reading  Frame 2 at t = 0.2  from outdir = /home/ajivani/AdvectionWL/_output
Frame 2 at time t = 0.2
    Reading  Frame 3 at t = 0.3  from outdir = /home/ajivani/AdvectionWL/_output
Frame 3 at time t = 0.3
    Reading  Frame 4 at t = 0.4  from outdir = /home/ajivani/AdvectionWL/_output
Frame 4 at time t = 0.4
    Reading  Frame 5 at t = 0.5  from outdir = /home/ajivani/AdvectionWL/_output
Frame 5 at time t = 0.5
    Reading  F